In [1]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
import numpy as np
from data_util import *
import threading
import multiprocessing

In [2]:
future = 10

def get_tecs(raw_dir = "stock_data/"):
    data_gen = DataGenerator(random=False, base_dir = raw_dir)
    data_gen.rewind()
    stateUtil = StateUtil(data_gen, future = future)
    data = []
    data_count = data_gen.max_steps()
    tec = TecAn()
    for i in tqdm(range(data_count)):
        raw = data_gen.next()
        price = raw[stateUtil.PRICE_KEY]
        amount = raw[stateUtil.AMOUNT_KEY]
        data.append([price, amount])
    
    def calcule_tecs():
        tecs = []
        print("Data {}".format(len(data)))
        df = pd.DataFrame(data, columns = ['Close', 'Volume'])
        close = df['Close']
        volume = df['Volume']
        for ta in tec.tas:
            value = ta(close, volume, 0.0, 0.0)
            tecs.append(value)
        return tecs
    
    return calcule_tecs()


In [3]:
def add_tacs(list, index, result):
    list = []
    #print(len(result[-1]))
    #print(index)
    for tec in result:
        list.append(tec.iloc[index])
    return list

def add_tacs_realtime(list, price, amount, tec):
    list = []
    list.extend(tec.add_ta(price, amount))
    #inds = np.array(tec.add_ta(price, amount))
    #list.extend(inds)
    #print(len(inds))
    #return np.array((np.array(list), inds), dtype=object)
    return list

In [4]:
def save(data_set, prefix = "", base_dir = "data/"):
    trainX = data_set[0]
    trainY = data_set[1]
    valX = data_set[2]
    valY = data_set[3]
    final_path = path + base_dir
    train_path = "{}{}trainX.npy".format(final_path, prefix)
    np.save(train_path, trainX)
    np.save("{}{}trainY.npy".format(final_path, prefix), trainY)
    np.save("{}{}valX.npy".format(final_path, prefix), valX)
    np.save("{}{}valY.npy".format(final_path, prefix), valY)
    print("Saving {} with {}".format(train_path, trainX.shape))


In [5]:
path = "./" 

windows = 30

import data_util
from data_generator import DataGenerator

def load_dataset(dir):
    load_datasets([dir])

def load_datasets(dirs, base_dir = "data/"):
    print(dirs)
    sets = []  
    for raw_dir in dirs:
        full_data = base_dir + raw_dir + "/"
        data_gen = DataGenerator(random = False, base_dir = full_data)
        data_gen.rewind()
        full = data_gen.steps
        tec = TecAn(windows = 20, windows_limit = 50)
        print("Processing {}".format(raw_dir))
        #print("Recovering TECs")
        #result = get_tecs(full_data)
        #print("TECs recovered {}".format((result[-1].size)))
        parsed = lambda list, price, amount, index: add_tacs_realtime(list, price, amount, tec)
        #parsed = lambda list, price, amount, index: add_tacs(list, index, result)
        print("Loading {} states".format(full))
        final_data = data_util.get_sets(data_gen,
                                        full, 
                                        val_percentage = 0.006, 
                                        path = full_data, 
                                        use_cache=False,
                                        on_state_parsed = parsed
                                       )
        count = len(final_data[0]) + len(final_data[-1])
        print("Total to be added {}".format(count))
        save(final_data, raw_dir)
        sets.append(final_data)
    return sets

def conc_sets(sets):
    trainX = sets[0][0]
    trainY = sets[0][1]
    valX = sets[0][2]
    valY = sets[0][3]
    for i in range(1,  len(sets)):
        data_set = sets[i]
        trainX = np.append(data_set[0], trainX, axis = 0)
        trainY = np.append(data_set[1], trainY, axis = 0)
        valX = np.append(data_set[2], valX, axis = 0)
        valY = np.append(data_set[3], valY, axis = 0)
    return trainX, trainY, valX, valY


#btcusd = load_datasets(["btcusd"])
#btcusd17 = load_datasets(["btcusd17"])
#btcusdAug19 = load_datasets(["btcusdAug19"])
#daiusd = load_datasets(["daiusd", "ethusd"])
#sets = load_datasets(["data/btceur", "data/btcusd", "data/daiusd", "data/ethusd", "data/ltcusd", "data/omgusd"])
#save(conc_sets(daiusd))



In [6]:
dirs = ["btceur", "btcusd", "daiusd", "ethusd", "ltcusd", "omgusd"]

sets = load_datasets(dirs)
save(conc_sets(sets))

['btceur', 'btcusd', 'daiusd', 'ethusd', 'ltcusd', 'omgusd']
Processing btceur
Loading 656864 states


  0%|          | 0/652922 [00:00<?, ?it/s]

  0%|          | 0/3941 [00:00<?, ?it/s]

Completed: (652922, 6) (652922, 2) (3941, 6) (3941, 2)
Total to be added 656863
Saving ./data/btceurtrainX.npy with (652922, 6)
Processing btcusd
Loading 388203 states


  0%|          | 0/385873 [00:00<?, ?it/s]

...

  0%|          | 0/3259 [00:00<?, ?it/s]

Completed: (539986, 6) (539986, 2) (3259, 6) (3259, 2)
Total to be added 543245
Saving ./data/ltcusdtrainX.npy with (539986, 6)
Processing omgusd
Loading 213410 states


  0%|          | 0/212129 [00:00<?, ?it/s]

  0%|          | 0/1280 [00:00<?, ?it/s]

Completed: (212129, 6) (212129, 2) (1280, 6) (1280, 2)
Total to be added 213409
Saving ./data/omgusdtrainX.npy with (212129, 6)
Saving ./data/trainX.npy with (2479552, 6)


In [7]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
import numpy as np


raw_dir = "ethusd/"
data_gen = DataGenerator(random=False, first_index=0, base_dir = raw_dir)
stateUtil = StateUtil(data_gen, future = 10)

FileNotFoundError: [Errno 2] No such file or directory: 'ethusd/'

In [ ]:
state = data_gen.next()
print(state['timestamp'])
state = data_gen.next()
print(state['timestamp'])

In [ ]:
state = data_gen.next()
print(state['timestamp'])
x, y, raw_state, furure_state = stateUtil.get_state(state, data_gen.index)

current = int(raw_state['timestamp'])
microcurrent = int(raw_state['microtimestamp'])

future = int(furure_state['timestamp'])
microfuture = int(furure_state['microtimestamp'])
print("{} - {} ".format(current, microcurrent))
print("{} - {} ".format(future, microfuture))
print("Diff: {}".format(future - current))
print("Y: {}".format(y))

ask = float(raw_state[stateUtil.ASKS_KEY][0][0]) 
print("Ask: {}".format(ask))
future_bid = float(furure_state[stateUtil.BIDS_KEY][0][0])
print("Bid: {}".format(future_bid))


In [ ]:
y

In [ ]:
stateUtil.onehot_encoded(0)

In [ ]:
path = "./data/"

trainX, trainY, positiveX, positiveY, negativeX, negativeY = load_data("ethusd", "train", path, balanced = True)

print("Loaded: {} {} ".format(trainX.shape, trainY.shape))

In [ ]:
get_gen(trainX, trainY, shuffle=False, time_steps = 30)

In [ ]:
def prepare_y(list):
    empty = np.array([[0, 0]])
    list = np.concatenate((empty, list), axis=0)
    list = np.delete(list , -1, 0)
    return list

In [ ]:
prepare_y(trainY).shape

In [ ]:
trainY.shape

In [ ]:
import time



def foo(bar, result, index):
    #print("hello {}".format(bar))
    if (index == 5):
        time.sleep(2.4)
    result[index] = "foo {}".format(index)

from threading import Thread

threads = [None] * 10
results = [None] * 10

for i in range(len(threads)):
    threads[i] = Thread(target=foo, args=('world!', results, i))
    threads[i].start()

# do some other stuff

for i in range(len(threads)):
    threads[i].join()

print(" ".join(results) )

In [ ]:
results = [None] * 11

In [ ]:
!pip install rx

In [ ]:
import rx
from rx import operators as ops
from rx import of, operators as op

test = of(1,2,3,4,5,6)
test1 = of(4,8,12,16,20)
test2 = of(5,10,15,20,25)
sub1 = test.pipe(
    op.zip(test1, test2)
)

test = sub1.run()

print(test)

In [ ]:
import rx
from rx import operators as ops
from rx import of, operators as op

test = of(1,2,3,4,5,6)
test1 = of(4,8,12,16,20)
test2 = of(5,10,15,20,25)

arrys = [test, test1, test2]

rxTest = of(arrys)

test = rx.zip(rxTest).run()

print(test)